In [1]:
import pandas as pd
from collections import Counter

ugene=pd.read_csv("/Applications/Unipro UGENE.app/Contents/MacOS/data/custom_annotations/plasmid_features.txt",sep="\t",header=1)
ugene['len']=ugene['Sequence'].apply(len)
ugene[ugene['Type ']=='Origin ']

,#Feature,Type,Sequence,len
249,2micron2_origin,Origin,ACGCATTTAAGCATAAACACGCACTATGCCGTTCTTCTCATGTATA...,1165
250,2micron_origin,Origin,TTATCGATGATAAGCTGTCAAAGATGAGAATTAATTCCACGGACTA...,1472
251,EBV_origin,Origin,CATTAAACGGCATGCAGGAAAAGGACAAGCAGCGAAAATTCACGCC...,2183
252,f1_origin,Origin,GCATTAAGCGCGGCGGGTGTGGTGGTTACGCGCAGCGTGACCGCTA...,307
253,oriT_RP4,Origin,gaataagggacagtgaagaaggaacacccgctcgcgggtgggccta...,99
254,p15A_origin,Origin,GCGCTAGCGGAGTGTATACTGGCTTACTATGTTGGCACTGATGAGG...,913
255,pBR322_origin,Origin,AGATCAAAGGATCTTCTTGAGATCCTTTTTTTCTGCGCGTAATCTG...,620
256,R6Kg_origin,Origin,CCATGTCAGCCGTTAAGTGTTCCTGTGTCACTCAAAATTGCTTTGA...,392
257,SV40_origin,Origin,ATCCCGCCCCTAACTCCGCCCAGTTCCGCCCATTCTCCGCCCCATG...,78


In [6]:
ugene['Type '].value_counts()

Feature        78
Primer         77
Gene           60
Promoter       34
Terminator     22
Regulatory     19
Origin          9
Name: Type , dtype: int64

In [155]:
import subprocess
from tempfile import NamedTemporaryFile
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd

def bash(inCommand, autoOutfile = True):
    if autoOutfile == True:
        tmp = NamedTemporaryFile()
        subprocess.call(inCommand +' > '+tmp.name, shell=True)
        f = open(tmp.name,'r')
        tmp.close()
        return(f.read())
    else:
        subprocess.call(inCommand, shell=True)  
def BLAST(seq, BLASTtype="x", flags = 'bitscore sseqid pident qstart qend sseq qseq sstart send length slen qlen'): 
    query = NamedTemporaryFile()
    SeqIO.write(SeqRecord(Seq(seq), id="temp"), query.name, "fasta")
    tmp = NamedTemporaryFile()
    bash(
        'blast'+BLASTtype+' -query ' + query.name + ' -out ' + tmp.name +
        #' -db ../BLAST_dbs/addgene_CDS_20-9-3' +
        ' -db /Users/mattmcguffie/database/BLAST_dbs/nr_db' +
        #' -db /Users/mattmcguffie/database/BLAST_dbs/fluors_20-9-2' +
        ' -culling_limit 10 ' +
        ' -max_target_seqs 1000 -outfmt "6 '+flags+'"')
    with open(tmp.name, "r") as file_handle:  #opens BLAST file
        align = file_handle.readlines()
        
    tmp.close() 
    query.close() 
    align=pd.DataFrame([ele.split() for ele in align],columns=flags.split())
    
    if not align.empty:
        #align['sseqid']=align['sseqid'].str.split("|",expand=True)[1]
        align['pident']=align['pident'].astype(float)
        align['slen']=align['slen'].astype(int)
        align['length']=align['length'].astype(int)
        align['qlen']=align['qlen'].astype(int)
        align['pmatch']=align['length']/((align['qlen']/3)-1)#because nuc->prot
        align['abspmatch']=1-abs(1-align['pmatch']) #because nuc->prot
        align['fractionmatch']=align['length']/align['slen']
        align['bitscore']=align['bitscore'].astype(float)
        align['score']=align['pident']*align['length']*align['fractionmatch']
        align=align.sort_values(by=['score'],ascending=False)
    return align 

In [153]:
csga="""atgAAACTTT TAAAAGTAGC AGCAATTGCA GCAATCGTAT TCTCCGGTAG CGCTCTGGCA
GGTGTTGTTC CTCAGTACGG CGGCGGCGGT AACCACGGTG GTGGCGGTAA TAATAGCGGC
CCAAATTCTG AGCTGAACAT TTACCAGTAC GGTGGCGGTA ACTCTGCACT TGCTCTGCAA
ACTGATGCCC GTAACTCTGA CTTGACTATT ACCCAGCATG GCGGCGGTAA TGGTGCAGAT
GTTGGTCAGG GCTCAGATGA CAGCTCAATC GATCTGACCC AACGTGGCTT CGGTAACAGC
GCTACTCTTG ATCAGTGGAA CGGCAAAAAT TCTGAAATGA CGGTTAAACA GTTCGGTGGT
GGCAACGGTG CTGCAGTTGA CCAGACTGCA TCTAACTCCT CCGTCAACGT GACTCAGGTT
GGCTTTGGTA ACAACGCGAC CGCTCATCAG TACtaa""".lower().replace(" ","").replace("\n","")

In [160]:
%%time
hits=BLAST(plas*2)
hits

CPU times: user 14.2 ms, sys: 7 ms, total: 21.2 ms
Wall time: 47.8 s


In [ ]:
blastx -query /Users/mattmcguffie/database/plasmids/plas_from_people/pbtk562.fasta -out ./test.blast -db ../BLAST_dbs/addgene_CDS_20-9-3 -culling_limit 10  -max_target_seqs 1000 -outfmt "6 bitscore sseqid pident qstart qend sseq qseq sstart send length slen qlen"

In [11]:
mTm=list(SeqIO.parse("/Users/mattmcguffie/database/plasmids/erm_gfp.fa", "fasta"))[0]
mTm=str(mTm.seq)

In [9]:
%time
BLAST(mTm)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


,bitscore,sseqid,pident,qstart,qend,sseq,qseq,sstart,send,length,slen,qlen,pmatch,abspmatch,fractionmatch,score
0,959.0,sp|P14752.1|,97.782,3829,5316,MNIPFVVETVLHDGLLKYKFKNSKIRSITTKPGKSKGAIFAYRSKK...,MNIPFVVETVLHDGLLKYKFKNSKIRSITTKPGKSKGAIFAYRSKS...,1,496,496,496,8961,0.166109,0.166109,1.000000,48499.872000
1,957.0,sp|P18629.1|,97.379,3829,5316,MNIPFVVETVLHDGLLKYKFKNSKIRSITTKPGKSKGAIFAYRSKK...,MNIPFVVETVLHDGLLKYKFKNSKIRSITTKPGKSKGAIFAYRSKS...,1,496,496,496,8961,0.166109,0.166109,1.000000,48299.984000
2,498.0,sp|P0A4D5.1|,100.000,1509,2243,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,1,245,245,245,8961,0.082050,0.082050,1.000000,24500.000000
3,494.0,sp|P21236.1|,99.184,1509,2243,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,1,245,245,245,8961,0.082050,0.082050,1.000000,24300.080000
4,494.0,sp|P20173.2|,99.184,1509,2243,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,MNKNIKYSQNFLTSEKVLNQIIKQLNLKETDTVYEIGTGKGHLTTK...,1,245,245,245,8961,0.082050,0.082050,1.000000,24300.080000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,57.4,sp|Q6GEH9.1|,41.975,1008,1244,KQFKTVKALILDNKVKDIIIATDAGREGELVARLILDKV---GNKK...,KQFKIVAELL--KQANTIIVATDSDREGENIAWSIIHKANAFSKDK...,82,159,81,711,8961,0.027127,0.027127,0.113924,387.338924
83,58.2,sp|Q4L8B8.1|,40.741,8902,8666,KQFKTVKSLILDKKVKEVIIATDAGREGELVARLILDKVH---NKK...,KQFKIVAELL--KQANTIIVATDSDREGENIAWSIIHKANAFSKDK...,82,159,81,711,8961,0.027127,0.027127,0.113924,375.951759
82,58.2,sp|Q4L8B8.1|,40.741,1008,1244,KQFKTVKSLILDKKVKEVIIATDAGREGELVARLILDKVH---NKK...,KQFKIVAELL--KQANTIIVATDSDREGENIAWSIIHKANAFSKDK...,82,159,81,711,8961,0.027127,0.027127,0.113924,375.951759
37,133.0,sp|P08956.3|,100.000,3,188,DLLTTGVDIPSICNIVFLRKVRSRILYEQMKGRATRLCPEVNKTSF...,DLLTTGVDIPSICNIVFLRKVRSRILYEQMKGRATRLCPEVNKTSF...,794,855,62,1170,8961,0.020764,0.020764,0.052991,328.547009


diamond blastx -d nr -q reads.fna -o matches.m8

In [39]:
%time
seq='''AATGATTTTATTTTGACTGATAGTGACCTGTTCGTTGCAACAAATTGATGAGCAATGCTTTTTTATAATG
CCAACTTTGTACAAAAAAGTTGGCACCATGAGGAGCCGGAGTAACTCCGGGGTCCGGCTGGACGGCTACG
CTCGACTGGTGCAACAGACCATCCTGTGCCATCAGAATCCAGTGACTGGCTTGCTTCCAGCCAGCTATGA
TCAGAAAGATGCTTGGGTCCGAGATAATGTGTACAGCATCTTGGCTGTGTGGGGTTTGGGCCTGGCCTAT
CGGAAGAATGCAGACCGGGATGAGGATAAGGCAAAGGCCTATGAATTGGAGCAGAGTGTAGTGAAGCTGA
TGAGAGGACTACTGCACTGCATGATCAGACAGGTGGATAAAGTAGAATCCTTCAAATATAGTCAGAGTAC
TAAGGATAGCCTCCATGCAAAGTACAACACCAAAACCTGTGCCACTGTAGTGGGTGATGATCAATGGGGA
CACCTGCAGTTGGATGCTACCTCTGTGTACCTGCTCTTCTTAGCCCAAATGACTGCCTCAGGACTCCATA
TCATCCACAGCCTAGATGAAGTCAATTTCATACAGAACCTTGTGTTTTACATTGAAGCTGCATATAAAAC
TGCTGACTTCGGGATATGGGAACGTGGAGACAAGACCAACCAAGGGATCTCAGAGTTGAATGCCAGTTCA
GTTGGAATGGCAAAGGCAGCCCTGGAAGCATTAGATGAACTGGATCTGTTTGGTGTGAAAGGTGGGCCTC
AATCAGTTATCCATGTCCTGGCTGATGAAGTACAGCACTGCCAGTCTATCCTAAATTCACTACTGCCCCG
TGCTTCACATCAAAGAGTGATGCTAGTCTACTCTCAGTGGTTTCCTTCCCTGCCTTGCAGTAGAGGG'''.lower().replace(" ","").replace("\n","")
hits=dBLAST(seq)
hits

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs


,bitscore,sseqid,pident,qstart,qend,sseq,sstart,send,length,slen,qlen,percmatch,fractionmatch,abs percmatch,pi_permatch,score
0,504.6,sp|P46020|KPB1_HUMAN,100.0,98,847,MRSRSNSGVRLDGYARLVQQTILCHQNPVTGLLPASYDQKDAWVRD...,1,250,250,1223,907,20.441537,0.204415,20.441537,20.441537,51.103843
2,502.3,sp|P18688|KPB1_RABIT,99.6,98,847,MRSRSNSGVRLDSYARLVQQTILCHQNPVTGLLPASYDQKDAWVRD...,1,250,250,1237,907,20.210186,0.202102,20.210186,20.129345,50.323363
1,502.7,sp|P18826|KPB1_MOUSE,99.6,98,847,MRSRSNSGVRLDGYARLVHQTILCHQNPVTGLLPASYDQKDAWVRD...,1,250,250,1241,907,20.145044,0.201450,20.145044,20.064464,50.161160
3,494.6,sp|Q64649|KPB1_RAT,98.4,98,847,MRSRSNSGVRLDSYARLVQQHTILCHQNPVTGLLPASYDQKDAWVR...,1,251,251,1242,907,20.209340,0.202093,20.209340,19.885990,49.913836
4,445.3,sp|P46018|KPB2_RABIT,86.8,98,847,MRSRSNSGVRLDGYARLVQQTILCYQNPVTGLLSASHEQKDAWVRD...,1,250,250,1235,907,20.242915,0.202429,20.242915,17.570850,43.927126
5,443.4,sp|Q8BWJ3|KPB2_MOUSE,86.4,98,847,MRSRSNSGVRLDGYARLVQQTILCYQNPVTGLLSASHDQKDAWVRD...,1,250,250,1235,907,20.242915,0.202429,20.242915,17.489879,43.724696
6,439.9,sp|P46019|KPB2_HUMAN,86.0,98,847,MRSRSNSGVRLDGYARLVQQTILCYQNPVTGLLSASHEQKDAWVRD...,1,250,250,1235,907,20.242915,0.202429,20.242915,17.408907,43.522267
7,430.3,sp|Q9W6R1|KPB2_TAKRU,84.0,98,847,MRSRSNSGVRLDGYARLVHETILGFQNPVTGLLPASVQKKDAWVRD...,1,250,250,1229,907,20.341741,0.203417,20.341741,17.087063,42.717657
8,361.7,sp|Q9W391|KPBA_DROME,70.0,98,856,MRSRSNSGVRLDYYQRIVHRLILAHQEPVTGLFPASNVNSHAWIRD...,1,253,253,1247,907,20.288693,0.202887,20.288693,14.202085,35.931275
9,285.8,sp|P34335|KPBA_CAEEL,56.4,98,856,MRSRSGSGVRLDRILFMVEQTICSHQNPITALFANQKDFPGHAWVR...,1,254,257,1226,907,20.962480,0.209625,20.962480,11.822838,30.384695


In [35]:
plas='''ggtcgaatgggcaggtagccggatcaagcgtatgcagccgccgcattgcatcagccatgatggatactttctcggcaggagcaaggtgggatgacaggagatcctgccccggcacttcgcccaatagcagccagtcccttcccgcttcagtgacaacgtcgagcacagctgcgcaaggaacgcccgtcgtggccagccacgatagccgcgctgcctcgtcctgcagttcattcagggcaccggacaggtcggtcttgacaaaaagaaccgggcgcccctgcgctgacagccggaacacggcggcatcagagcagccgattgtctgttgtgcccagtcatagccgaatagcctctccacccaagcggccggagaacctgcgtgcaatccatcttgttcaatcatgcgaaacgatcctcatcctgtctcttgatcagatcttgatcccctgcgccatcagatccttggcggcaagaaagccatccagtttactttgcagggcttcccaaccttaccagagggcgccccagctggcaattccggttcgcttgctgtccataaaaccgcccagtctagctatcgccatgtaagcccactgcaagctacctgctttctctttgcgcttgcgttttcccttgtccagatagcccagtagctgacattcatccggggtcagcaccgtttctgcggactggctttctacgtgttccgcttcctttagcagcccttgcgccatgcatttgttaatcgaattatccattgctgttgacaaagggaatcaggggatcttgaagttcctattccgaagttcctattctctagaaagtataggaacttcaGAGCGcTTTTGAAGcTaattcGAgatcatgcatgagctcactagtggatcccccgggctgcaggaattcctcgagaagcttgggcccggtacctcgcgaaggccttgcaggccaaccagataagtgaaatctagttccaaactattttgtcatttttaattttcgtattagcttacgacgctacacccagttcccatctattttgtcactcttccctaaataatccttaaaaactccatttccacccctcccagttcccaactattttgtccgcccacagcggggcatttttcttcctgttatgtttgggcgctgcattaatgaatcggccaacgcgcggggagaggcggtttgcgtattgggcgctcttccgcttcctcgctcactgactcgctgcgctcggtcgttcggctgcggcgagcggtatcagctcactcaaaggcggtaatacggttatccacagaatcaggggataacgcaggaaagaacatggcggcagttcaacctgttgatagtacgtactaagctctcatgtttcacgtactaagctctcaagtttaacgtactaagctctcatgtttaacgaactaaaccctcatggctaacgtactaagctctcatggctaacgtactaagctctcatgtttcacgtactaagctctcatgtttgaacaataaaattaatataaatcagcaacttaaatagcctctaaggttttaagttttataagaaaaaaaagaatatataaggcttttaaagcttttaaggtttaacggttgtggacaacaagccagggatgtaacgcactgagaagcccttagagcctctcaaagcaattttgagtgacacaggaacacttaacggctgacaaccggctggtaacaggattagcagagcgaggtatgtaggcggtgctacagagttcttgaagtggtggcctaactacggctacactagaaggacagtatttggtatctgcgctctgctgaagccagttaccttcggaaaaagagttggtagctcttgatccggcaaacaaaccaccgctggtagcggtggtttttttgtttgcaagcagcagattacgcgcagaaaaaaaggatctcaagaagatcctttgatcttttctacggggtctgacgctcagtggaacgaaaactcacgttaagggattttggtcatgagattatcaaaaaggatcttcacctagatccttttaaattaaaaatgaagttttaaatcaatctaaagtatatatgagtaaacttggtctgacagttaccaatgcttaatcagtgaggcacctatctcagcgatctgtctatttcgttcatccatagttgcctgactccccgtcgtgtagataactacgatacgggagggcttaccatctggccccagtgctgcaatgataccgcgagacccacgctcaccggctccagatttatcagcaataaaccagccagccggaagggccgagcgcagaagtggtcctgcaactttatccgcctccatccagtctattaattgttgccgggaagctagagtaagtagttcgccagttaatagtttgcgcaacgttgttgccattgctacaggcatcgtggtgtcacgctcgtcgtttggtatggcttcattcagctccggttcccaacgatcaaggcgagttacatgatcccccatgttgtgcaaaaaagcggttagctccttcggtcctccgatcgttgtcagaagtaagttggccgcagtgttatcactcatggttatggcagcactgcataattctcttactgtcatgccatccgtaagatgcttttctgtgactggtgagtactcaaccaagtcattctgagaatagtgtatgcggcgaccgagttgctcttgcccggcgtcaatacgggataataccgcgccacatagcagaactttaaaagtgctcatcattggaaaacgttcttcggggcgaaaactctcaaggatcttaccgctgttgagatccagttcgatgtaacccactcgtgcacccaactgatcttcagcatcttttactttcaccagcgtttctgggtgagcaaaaacaggaaggcaaaatgccgcaaaaaagggaataagggcgacacggaaatgttgaatactcatactcttcctttttcaatattattgaagcatttatcagggttattgtctcatgagcggatacatatttgaatgtatttagaaaaataaacaaataggggttccgcgcacatttccccgaaaagtgccacctgacgtctaagaaaccattattatcatgacattaacctataaaaataggcgtatcacgaggccctttcgtctcgcgcgtttcggtgatgacggtgaaaacctctgacacatgcagctcccggagacggtcacagcttgtctgtaagcggatgccgggagcagacaagcccgtcagggcgcgtcagcgggtgttggcgggtgtcggggctggcttaactatgcggcatcagagcagattgtactgagagtgcaccatatgcggtgtgaaataccgcacagatgcgtaaggagaaaataccgcatcaggcgccattcgccattcaggctgcgcaactgttgggaagggcgatcggtgcgggcctcttcgctattacgccagctagaggaccagccgcgtaacctggcaaaatcggttacggttgagtaataaatggatgccctgcgtaagcgggtgtgggcggacaataaagtcttaaactgaacaaaatagatctaaactatgacaataaagtcttaaactagacagaatagttgtaaactgaaatcagtccagttatgctgtgaaaaagcatactggacttttgttatggctaaagcaaactcttcattttctgaagtgcaaattgcccgtcgtattaaagaggggcgtggggttcgaggtcgacggtatcgataagctagcttaattagctgagcttggactcctgttgatagatccagtaatgacctcagaactccatctggatttgttcagaacgctcggttgccgccgggcgttttttattggtgagaatccaagctagactgcgatgagttatcaaaaagagtattgacttaaagtctaacctataggatacttacagccatcgagagggacacggcgaggaattgtgagcggataacaattccatacagaaacagaggagatattacatatgagtaaaggagaagagcttttcacaggagttgtcccaatcctcgtggaattagacggtgatgttaatgggcacaagttctctgtcagtggagagggtgaaggcgacgcaacatatggcaagctgacccttaaatttatttgcaccacgggtaaactacctgttccatggccaacactggtcactacgttcgggtatggggttcagtgctttgcgcgctacccagatcacatgaaacagcacgactttttcaagagtgcaatgcccgaaggctatgtacaggagagaaccatcttttttaaggatgacggcaactataagacacgcgccgaagtgaagttcgagggtgatacccttgttaatagaatcgagttaaagggtattgactttaaggaagatggaaatattttaggccacaaactggaatataactataactcccataatgtgtacattatggccgacaagcaaaagaacggtatcaaggttaacttcaagatcagacacaacattgaggatggaagcgttcaactagccgaccattaccaacaaaacaccccaattggcgatgggcctgtgctgttaccagacaaccattacctgtccactcaatctgccctttcgaaagatcccaacgaaaagcgcgaccacatggtccttcttgagtttgtcacggctgctgggattacacacggcatggatgaactatacaaataaatccgtaatcgttaatccgcaaataacgtaaaaacccgcttcggcgggtttttttatggggggagtttagggaaagagcatttgtcagtaatgactctctagcttgaggcatcaaataaaacgaaaggctcagtcgaaagactgggcctttcgttttatctgttgtttgtcggtgaacgctctcctgagtaggacaaatccgccgctaggagcttgcggcccggacgaTCtcgaattGggGATCTtgaagtAcctattccgaagttcctattctctagaaagtataggaacttcagagcgcttttgaagctgatgtgcttaaaaacttactcaatggaataattCTAGaTAATTCACTagtgattggaaatgttggatcctcatactcttcctttttcaattcagaagaactcgtcaagaaggcgatagaaggcgatgcgctgcgaatcgggagcggcgataccgtaaagcacgaggaagcggtcagcccattcgccgccaagctcttcagcaatatcacgggtagccaacgctatgtcctgatagcggtccgccacacccagccggccacagtcgatgaatccagaaaagcggccattttccaccatgatattcggcaagcaggcatcgccatgggtcacgacgagatcctcgccgtcgggcatgcgcgccttgagcctggcgaacagttcggctggcgcgagcccctgatgctcttcgtccagatcatcctgatcgacaagaccggcttccatccgagtacgtgctcgctcgatgcgatgtttcgcttggt'''

In [227]:
import subprocess
from tempfile import NamedTemporaryFile
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd
import numpy as np

def DIAMOND(seq, db='/Users/mattmcguffie/Desktop/uniprot/swissprot.dmnd', flags = 'sseqid pident qstart qend sstart send length slen qlen'): 
    query = NamedTemporaryFile()
    SeqIO.write(SeqRecord(Seq(seq), id="temp"), query.name, "fasta")
    tmp = NamedTemporaryFile()
    
    subprocess.call(f'diamond blastx -d {db} -q {query.name} -o {tmp.name} --outfmt 6 {flags}',shell=True)
    
    with open(tmp.name, "r") as file_handle:  #opens BLAST file
        align = file_handle.readlines()
        
    tmp.close() 
    query.close() 
    align=pd.DataFrame([ele.split() for ele in align],columns=flags.split())
    align=align.apply(pd.to_numeric, errors='ignore')
    if not align.empty:
        align['qlen'] = (align['qlen']/2).astype('int')
        align['aa_length'] = align['length']
        align['percmatch'] = (align['aa_length']/align['slen']*100)
        align['abs percmatch'] = 100-abs(100-align['percmatch'])#eg changes 102.1->97.9
        align['pi_permatch'] = (align["pident"]*align["abs percmatch"])/100
        align['sframe'] = (align['qstart']<align['qend']).astype(int).replace(0,-1)
        #align['sframe'] = (align['qframe']>0).astype(int).replace(0,-1)
        align['s'], align['e'] = align[['qstart','qend']].min(axis=1), align[['qstart','qend']].max(axis=1)
        align['length'] = (align['e']+1)-align['s']
        align['length'] = abs(align['qend']-align['qstart'])+1
        align['score'] = (align['pi_permatch']/100)*align["length"]
        align['fragment'] = align["percmatch"] < 95
        
        align = align.sort_values(by=['score'],ascending=False)
        
        wiggleSize = 0.15 #this is the percent "trimmed" on either end eg 0.1 == 90%
        align['wiggle'] = (align['length'] * wiggleSize).astype(int)
        align['wstart'] =  align['qstart'] + align['wiggle']
        align['wend']   =  align['qend']   - align['wiggle']

        align = align.sort_values(by=["score","length","percmatch"], ascending=[False, False, False])

        #subtracts a full plasLen if longer than tot length
        align['qstart'] = np.where(align['qstart'] >= align['qlen'], align['qstart'] - align['qlen'], align['qstart'])    
        align['qend']   = np.where(align['qend']   >= align['qlen'], align['qend']   - align['qlen'], align['qend'])

        align['wstart'] = np.where(align['wstart'] >= align['qlen'], align['wstart'] - align['qlen'], align['wstart'])    
        align['wend']   = np.where(align['wend']   >= align['qlen'], align['wend']   - align['qlen'], align['wend'])

        align = align.drop_duplicates(subset=align.columns.difference(['qframe']))
        align = align.reset_index(drop=True)

    return align 

In [228]:
df=DIAMOND(plas*2)#, db = "/Users/mattmcguffie/Documents/GitHub/pLannotate/data/fpbase.dmnd")
df=df[df['pident']>95]
df

,sseqid,pident,qstart,qend,sstart,send,length,slen,qlen,aa_length,percmatch,abs percmatch,pi_permatch,sframe,score,fragment,wiggle,wstart,wend
0,sp|P62593|BLAT_ECOLX,99.3,2977,2120,1,286,858,286,5551,286,100.000000,100.000000,99.300000,-1,851.994000,False,128,3105,1992
1,sp|P62594|BLAT_SALTI,99.3,2977,2120,1,286,858,286,5551,286,100.000000,100.000000,99.300000,-1,851.994000,False,128,3105,1992
2,sp|Q48406|BLAT_KLEOX,99.0,2977,2120,1,286,858,286,5551,286,100.000000,100.000000,99.000000,-1,849.420000,False,128,3105,1992
3,sp|P00552|KKA2_KLEPN,100.0,403,5163,1,264,792,264,5551,264,100.000000,100.000000,100.000000,-1,792.000000,False,118,521,5045
4,sp|P42212|GFP_AEQVI,99.2,4042,4755,1,238,714,238,5551,238,100.000000,100.000000,99.200000,1,708.288000,False,107,4149,4648
25,sp|P00552|KKA2_KLEPN,100.0,403,2,1,134,402,264,5551,134,50.757576,50.757576,50.757576,-1,204.045455,True,60,463,-58
26,sp|P00552|KKA2_KLEPN,100.0,5549,5163,136,264,387,264,5551,129,48.863636,48.863636,48.863636,-1,189.102273,True,58,5607,5105


In [741]:
df.loc[df['pi_permatch']==100]['pi_permatch'] = df.loc[df['pi_permatch']==100]['pi_permatch']*1.5
df

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,sseqid,pident,qstart,qend,sstart,send,length,slen,qlen,aa_length,...,pi_permatch,sframe,score,fragment,wiggle,wstart,wend,"(t, e)",t,e
0,sp|P62593|BLAT_ECOLX,99.3,2977,2120,1,286,858,286,5551,286,...,99.300000,-1,851.994000,False,128,3105,1992,f,f,f
1,sp|P62594|BLAT_SALTI,99.3,2977,2120,1,286,858,286,5551,286,...,99.300000,-1,851.994000,False,128,3105,1992,f,f,f
2,sp|Q48406|BLAT_KLEOX,99.0,2977,2120,1,286,858,286,5551,286,...,99.000000,-1,849.420000,False,128,3105,1992,f,f,f
3,sp|P00552|KKA2_KLEPN,100.0,403,5163,1,264,792,264,5551,264,...,100.000000,-1,792.000000,False,118,521,5045,f,f,f
4,sp|P42212|GFP_AEQVI,99.2,4042,4755,1,238,714,238,5551,238,...,99.200000,1,708.288000,False,107,4149,4648,f,f,f
25,sp|P00552|KKA2_KLEPN,100.0,403,2,1,134,402,264,5551,134,...,50.757576,-1,204.045455,True,60,463,-58,f,f,f
26,sp|P00552|KKA2_KLEPN,100.0,5549,5163,136,264,387,264,5551,129,...,48.863636,-1,189.102273,True,58,5607,5105,f,f,f


In [743]:
df

,sseqid,pident,qstart,qend,sstart,send,length,slen,qlen,aa_length,...,pi_permatch,sframe,score,fragment,wiggle,wstart,wend,"(t, e)",t,e
0,sp|P62593|BLAT_ECOLX,99.3,2977,2120,1,286,858,286,5551,286,...,99.300000,-1,851.994000,False,128,3105,1992,f,f,f
1,sp|P62594|BLAT_SALTI,99.3,2977,2120,1,286,858,286,5551,286,...,99.300000,-1,851.994000,False,128,3105,1992,f,f,f
2,sp|Q48406|BLAT_KLEOX,99.0,2977,2120,1,286,858,286,5551,286,...,99.000000,-1,849.420000,False,128,3105,1992,f,f,f
3,sp|P00552|KKA2_KLEPN,100.0,403,5163,1,264,792,264,5551,264,...,100.000000,-1,1188.000000,False,118,521,5045,f,f,f
4,sp|P42212|GFP_AEQVI,99.2,4042,4755,1,238,714,238,5551,238,...,99.200000,1,708.288000,False,107,4149,4648,f,f,f
25,sp|P00552|KKA2_KLEPN,100.0,403,2,1,134,402,264,5551,134,...,50.757576,-1,204.045455,True,60,463,-58,f,f,f
26,sp|P00552|KKA2_KLEPN,100.0,5549,5163,136,264,387,264,5551,129,...,48.863636,-1,189.102273,True,58,5607,5105,f,f,f


In [25]:
from io import StringIO
from Bio import SeqIO
import Bio.SwissProt as sp
import requests
import pandas as pd

uniprot='P42212'
url = f'https://www.uniprot.org/uniprot/{uniprot}.txt'
r = requests.get(url, allow_redirects=False)
embl_str=r.text

record_dict=SeqIO.to_dict(SeqIO.parse(StringIO(embl_str), 'swiss'))
swiss = record_dict[list(record_dict.keys())[0]]

#this is all wonky
attrs = ["annotations,gene_name","description","annotations,comment","annotations,protein_existence"]
details = {"name":swiss.name}
for attr in attrs:
    attr=attr.split(",")
    try:
        if len(attr)==1:
            details[attr[1]] = getattr(swiss,attr[0])
        elif len(attr)==2:
            details[attr[1]] = getattr(swiss,attr[0])[attr[1]]
    except:
        pass

#pd.DataFrame([ele.split(":",1) for ele in details['comment'].split("\n")]).set_index(0).T
comment=[ele.split(":",1) for ele in details['comment'].split("\n")]
comment={ele[0]:ele[1].strip() for ele in comment}
comment['FUNCTION']

'Energy-transfer acceptor. Its role is to transduce the blue chemiluminescence of the protein aequorin into green fluorescent light by energy transfer. Fluoresces in vivo upon receiving energy from the Ca(2+)-activated photoprotein aequorin.'

In [26]:
details

{'name': 'GFP_AEQVI',
 'gene_name': 'Name=GFP;',
 'comment': 'FUNCTION: Energy-transfer acceptor. Its role is to transduce the blue chemiluminescence of the protein aequorin into green fluorescent light by energy transfer. Fluoresces in vivo upon receiving energy from the Ca(2+)-activated photoprotein aequorin.\nBIOPHYSICOCHEMICAL PROPERTIES: Absorption:   Abs(max)=395 nm;   Note=Exhibits a smaller absorbance peak at 470 nm. The fluorescence   emission spectrum peaks at 507-510 nm with a shoulder at 545 nm   (PubMed:8137953, PubMed:9154981). The exact value of the emission   maximum depends on the environment of the chromophore   (PubMed:10220315). As a consequence, mutant versions have been   designed that have substantially shifted emission spectra, including   yellow-emission variants (YFP), blue and cerulean fluorescing   proteins (PubMed:9145105, PubMed:9782051, PubMed:17685554).   {ECO:0000269|PubMed:10220315, ECO:0000269|PubMed:17685554,   ECO:0000269|PubMed:8137953, ECO:0000269

In [23]:
comment

{'FUNCTION': "Part of the relaxosome complex that is responsible for plasmid transfer during conjugation. Locally unwinds DNA and catalyzes the cleavage of one of the DNA strands at oriT. The cleaved strand is then transferred through the dedicated type IV secretion apparatus. MobA remains covalently linked at the 5' end of the strand, and once in the recipient cell, it probably catalyzes the rejoining of the two ends of the strand, re-forming the circular plasmid DNA. The primase activity of MobA is essential for the synthesis of primers that will initiate the DNA replication events necessary to form the double- stranded plasmid in the recipient cell. {ECO:0000269|PubMed:1738602, ECO:0000269|PubMed:8955311}.",
 'CATALYTIC ACTIVITY': 'Reaction=ATP-independent breakage of single-stranded DNA, followed by   passage and rejoining.; EC=5.6.2.1;',
 'COFACTOR': 'Name=Mg(2+); Xref=ChEBI:CHEBI:18420;   Evidence={ECO:0000269|PubMed:1738602, ECO:0000269|PubMed:8223650}; Name=Mn(2+); Xref=ChEBI:C

In [450]:
uni=list(SeqIO.parse("/Users/mattmcguffie/Desktop/uniprot/uniprot_sprot.fasta", "fasta"))

In [505]:
trim = [ele for ele in uni if "PE=4" not in ele.description and "PE=5" not in ele.description]

In [508]:
SeqIO.write(trim,'/Users/mattmcguffie/Desktop/uniprot/trimmed_uniprot_sprot.fasta', 'fasta')

547899

In [747]:
import subprocess
from tempfile import NamedTemporaryFile
def bash(inCommand, autoOutfile = True):
    if autoOutfile == True:
        tmp = NamedTemporaryFile()
        subprocess.call(inCommand +' > '+tmp.name, shell=True)
        f = open(tmp.name,'r')
        tmp.close()
        return(f.read())
    else:
        subprocess.call(inCommand, shell=True)  

In [754]:
out=bash("cd-hit")
print(out)

		====== CD-HIT version 4.8.1 (built on Aug 15 2020) ======

Usage: cd-hit [Options] 

Options

   -i	input filename in fasta format, required, can be in .gz format
   -o	output filename, required
   -c	sequence identity threshold, default 0.9
 	this is the default cd-hit's "global sequence identity" calculated as:
 	number of identical amino acids or bases in alignment
 	divided by the full length of the shorter sequence
   -G	use global sequence identity, default 1
 	if set to 0, then use local sequence identity, calculated as :
 	number of identical amino acids or bases in alignment
 	divided by the length of the alignment
 	NOTE!!! don't use -G 0 unless you use alignment coverage controls
 	see options -aL, -AL, -aS, -AS
   -b	band_width of alignment, default 20
   -M	memory limit (in MB) for the program, default 800; 0 for unlimitted;
   -T	number of threads, default 1; with 0, all CPUs will be used
   -n	word_length, default 5, see user's guide for choosing it
   -l	length of thr